In [1]:
import copy
import os
import pickle
import random
import time

import folium
import matplotlib.cm as cm
import networkx as nx
import numpy as np
import osmnx as ox
import pulp as pl
import requests
from dotenv import dotenv_values
from geopy.geocoders import Nominatim
from haversine import Unit, haversine_vector

ox.config(use_cache=True, log_console=True)

/home/kind/Works/personal/bk-imp/.venv/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


# Generate Warehouses and Stores

In [ ]:
## Create OSM map of HCMC
if not os.path.exists("ho_chi_minh_city.graphml"):
    graph = ox.graph_from_place(
        "Ho Chi Minh City", network_type="drive", simplify=False
    )
    ox.save_graphml(graph, filepath="ho_chi_minh_city.graphml")
else:
    graph = ox.io.load_graphml("ho_chi_minh_city.graphml")

In [ ]:
fig, ax = ox.plot_graph(graph)

In [ ]:
# Obtain a list of nodes from the graph
nodes = list(graph.nodes)

chosen_locations = set()
# Generate random warehouses within the graph
warehouses = []
while len(warehouses) < 10000:
    # Select a random node
    random_node = random.choice(nodes)
    lat, long = graph.nodes[random_node]["y"], graph.nodes[random_node]["x"]
    warehouses.append((random_node, lat, long))
    chosen_locations.add(random_node)
with open("warehouses.pkl", "wb") as f:
    pickle.dump(warehouses, f)

# Generate random stores within the graph
stores = []
while len(stores) < 10000:
    # Select a random node
    random_node = random.choice(nodes)
    if random_node not in chosen_locations:
        lat, long = (
            graph.nodes[random_node]["y"],
            graph.nodes[random_node]["x"],
        )
        stores.append((random_node, lat, long))
with open("stores.pkl", "wb") as f:
    pickle.dump(stores, f)

# Direct Distance
In this approach, we make the assumption that the distance between two shops is independent of the road network.

In [ ]:
M = 100  # Number of warehouses
N = 1000  # Number of stores

sampled_warehouses = random.choices(warehouses, k=M)
sampled_stores = random.choices(stores, k=N)

## Calculate Distance Matrix

In [ ]:
dinstance_matrix = haversine_vector(
    [(store[1], store[2]) for store in sampled_stores],
    [(warehouse[1], warehouse[2]) for warehouse in sampled_warehouses],
    Unit.KILOMETERS,
    comb=True,
)

In [ ]:
dinstance_matrix.shape

## Linear Programming

In [ ]:
# Set up the problem
problem = pl.LpProblem("Warehouse_Optimization", pl.LpMinimize)

### Define the environment

In [ ]:
warehouse_ids = range(M)
w_cost = random.sample(range(100, 201), M)  # Operational cost of warehouses
w_capacity = random.sample(
    range(13000, 15001), M
)  # Operational cost of warehouses
store_ids = range(N)
demands = random.sample(range(10, 1101), N)  # Daily demand of stores
d_cost = copy.copy(
    dinstance_matrix
)  # Delivery cost between warehouses vs stores

In [ ]:
assert sum(w_capacity) > sum(
    demands
), "All warehouses' capacity is less than stores' demands"

### Define the decision variables

In [ ]:
wr = pl.LpVariable.dicts(
    "warehouse_store_connection", (warehouse_ids, store_ids), cat="Binary"
)

### Define the objective function

In [ ]:
total_cost = pl.lpSum(
    w_cost[w] * wr[w][s] + d_cost[w][s] * wr[w][s]
    for w in warehouse_ids
    for s in store_ids
)
problem += total_cost

### Define the constraints

In [ ]:
# Each store must be assigned to one warehouse
for i in store_ids:
    problem += pl.lpSum(wr[j][i] for j in warehouse_ids) == 1

In [ ]:
# Total demand of assigned stores for a warehouse must be smaller than its capacity
for w in warehouse_ids:
    problem += (
        pl.lpSum(demands[s] * wr[w][s] for s in store_ids) <= w_capacity[w]
    )

## Solve the problem!!!

### COIN-OR Branch and Cut solver (CBC)
https://coin-or.github.io/Cbc/intro.html#:~:text=The%20COIN%2DOR%20Branch%20and,executable%20version%20is%20also%20available.

In [ ]:
problem.solve(pl.PULP_CBC_CMD(timeLimit=120, threads=6, msg=0))

In [ ]:
# Print the total cost
print("Total Cost: ", pl.value(problem.objective))

### CPLEX Solver
Get it free from here https://www.ibm.com/academic/topic/data-science

or https://storage.googleapis.com/thaitang-sharing/cplex_studio2211.linux_x86_64.bin

In [ ]:
os.environ["CPLEX_HOME"] = "/opt/ibm/ILOG/CPLEX_Studio2211/cplex"
os.environ["CPO_HOME"] = "/opt/ibm/ILOG/CPLEX_Studio2211/cpoptimizer"
os.environ["PATH"] += (
    ":"
    + os.environ["CPLEX_HOME"]
    + "/bin/x86-64_linux:"
    + os.environ["CPO_HOME"]
    + "/bin/x86-64_linux"
)
os.environ["LD_LIBRARY_PATH"] += (
    ":"
    + os.environ["CPLEX_HOME"]
    + "/bin/x86-64_linux:"
    + os.environ["CPO_HOME"]
    + "/bin/x86-64_linux"
)
os.environ[
    "PYTHONPATH"
] = "/opt/ibm/ILOG/CPLEX_Studio2211/cplex/python/3.10/x86-64_linux"

In [ ]:
problem.solve(pl.CPLEX_CMD(timeLimit=120, threads=6, msg=0))

In [ ]:
# Print the total cost
print("Total Cost: ", pl.value(problem.objective))

# Actual Distance
Actual distance is calculated by calling the API of OpenStreetMap.

Note that the Openrouteservice API has a limit of 20 requests per minute for Isochrones.

In [ ]:
def calculate_distance(graph, source_node, target_node):
    return (
        nx.shortest_path_length(
            graph, source_node, target_node, weight="length"
        )
        / 1000
    )

In [ ]:
sampled_warehouses[0]

In [ ]:
sampled_stores[10]

In [ ]:
print(
    calculate_distance(graph, sampled_warehouses[0][0], sampled_stores[10][0])
)

In [ ]:
# Update the actual distance from warehouse to assigned store
updated_cost = np.full_like(d_cost, False)
for w in warehouse_ids:
    for s in store_ids:
        if (pl.value(wr[w][s]) == 1) & (not updated_cost[w][s]):
            new_cost = calculate_distance(
                graph,
                sampled_warehouses[w][0],
                sampled_stores[s][0],
            )
            print(
                "Update cost from warehouse {} to store {} - {} to {}".format(
                    w, s, round(d_cost[w][s], 2), round(new_cost, 2)
                )
            )
            d_cost[w][s] = new_cost
            updated_cost[w][s] = True
            break

# Analyze Results

In [3]:
with open("results/exp1/warehouses.pkl", "rb") as f:
    warehouses = pickle.load(f)
with open("results/exp1/stores.pkl", "rb") as f:
    stores = pickle.load(f)
with open("results/exp1/wr.pkl", "rb") as f:
    wr = pickle.load(f)

In [4]:
# Create a map centered around the first warehouse
map_obj = folium.Map(
    location=[10.779748918558694, 106.6990408245344], zoom_start=14
)

# Color map
colors = [c for c in list(folium.Icon.color_options) * 10 if c not in ["black", "white"]]
color_maps = []

# Add markers for warehouses
counter = 0
for w, warehouse in enumerate(warehouses):
    num_assigned_stores = 0
    for s, store in enumerate(stores):
        if pl.value(wr[w][s]) == 1:
            num_assigned_stores += 1
    counter += num_assigned_stores
    print(w, num_assigned_stores)
print(counter)
#             color_maps.append([w, colors.pop()])
#             hex_color = [c for c in color_maps if c[0] == w][0][1]
#             folium.Marker(
#                 [warehouse[1], warehouse[2]],
#                 popup="Warehouse {}".format(w),
#                 icon=folium.Icon(
#                     color=hex_color,
#                     icon_color="white",
#                     icon="car",
#                     prefix="fa"
#                 ),
#             ).add_to(map_obj)
#             counter += 1
#             break
# print("Number of warehouses:", counter)

# # Add markers for stores
# for w, warehouse in enumerate(warehouses):
#     for s, store in enumerate(stores):
#         if pl.value(wr[w][s]) == 1:
#             hex_color = [c for c in color_maps if c[0] == w][0][1]
#             folium.Marker(
#                 [store[1], store[2]],
#                 popup="Store assigned to Warehouse {}".format(w),
#                 icon=folium.Icon(
#                     color=hex_color,
#                     icon_color=hex_color,
#                     icon="shopping-cart",
#                     prefix="fa",
#                 ),
#             ).add_to(map_obj)

# # Display the map
# map_obj

0 22
1 14
2 10
3 20
4 12
5 22
6 53
7 17
8 16
9 28
10 10
11 20
12 10
13 25
14 23
15 26
16 10
17 19
18 11
19 12
20 10
21 37
22 23
23 17
24 10
25 12
26 15
27 11
28 18
29 23
30 25
31 27
32 15
33 10
34 31
35 15
36 32
37 10
38 41
39 38
40 22
41 21
42 22
43 13
44 44
45 19
46 22
47 10
48 17
49 10
1000
